In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
diag = pd.read_csv('eicu_full/diagnosis.csv')[['patientunitstayid', 'icd9code']]

diag.dropna(axis=0, inplace=True)

In [3]:
split_codes = diag['icd9code'].str.split(', ', expand=True)

diag['icd9'] = split_codes.iloc[:, 0]

diag.drop('icd9code', inplace=True, axis=1)
diag

,patientunitstayid,icd9
0,141168,414.00
2,141168,491.20
3,141168,491.20
4,141168,428.0
5,141168,427.31
...,...,...
2710667,3353251,599.0
2710668,3353251,038.9
2710669,3353254,584.9
2710670,3353254,578.9


In [4]:
def decimal_format(x):
    try:
        return '{:.2f}'.format(float(x)).zfill(6)
    except:
        return x


def padding_icd9(df, column):
    """
    Padding ICD9 codes to 3 digits left of decimal and 2 right of decimal
    """
    df[column] = df[column].apply(lambda x: decimal_format(x))
    return df

diag_pad = padding_icd9(diag, 'icd9')

In [5]:
hyper_labels = pd.read_csv('diagnosis_hyperkalemia.csv', names=['icd9', 'name', 'hyperkalemia', 'source'])
hyper_labels.drop(['name', 'source'], axis=1, inplace=True)
hyper_labels.drop_duplicates(['icd9'], inplace=True)
hyper_labels['hyperkalemic'] = hyper_labels['hyperkalemia'] == 1
hyper_labels.drop(['hyperkalemia'], axis=1, inplace=True)
hyper_labels = padding_icd9(hyper_labels, 'icd9')
hyper_labels = hyper_labels.set_index('icd9')
hyper_labels

,hyperkalemic
icd9,
008.45,False
009.10,True
038.90,True
053.90,False
078.50,False
...,...
996.85,False
997.10,False
998.11,True


In [6]:
diag_pad

,patientunitstayid,icd9
0,141168,414.00
2,141168,491.20
3,141168,491.20
4,141168,428.00
5,141168,427.31
...,...,...
2710667,3353251,599.00
2710668,3353251,038.90
2710669,3353254,584.90
2710670,3353254,578.90


In [7]:
joined = diag_pad.join(hyper_labels, on='icd9', how='left')
joined

,patientunitstayid,icd9,hyperkalemic
0,141168,414.00,False
2,141168,491.20,False
3,141168,491.20,False
4,141168,428.00,True
5,141168,427.31,False
...,...,...,...
2710667,3353251,599.00,False
2710668,3353251,038.90,True
2710669,3353254,584.90,True
2710670,3353254,578.90,False


In [8]:
joined.isnull().sum()

patientunitstayid         0
icd9                      0
hyperkalemic         258625
dtype: int64

In [9]:
joined.dropna(axis=0, inplace=True)

joined['hyperkalemic'] = joined['hyperkalemic'].astype(int)

joined.drop('icd9', axis=1, inplace=True)

In [10]:
squashed = joined.groupby('patientunitstayid').sum()

squashed['hyperkalemic'] = squashed['hyperkalemic'] > 0